In [43]:
from neo4j import GraphDatabase
import os 
import dotenv   
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

dotenv.load_dotenv()

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://c5653eb6.databases.neo4j.io"
USERNAME = os.getenv("NEO4J_USERNAME")
PASSWORD = os.getenv("NEO4J_PASSWORD")

def execute_cypher_query(statements: List[str]):
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
    """
    Execute a Cypher query against a Neo4j database.
    
    Args:
        query: The Cypher query to execute
        
    Returns:
        For MATCH queries: List of dictionaries containing the query results
        For other queries: None
    """
    results = []
    
    try:
        with driver.session() as session:
            for statement in statements:
                # Check if this is a read query (starts with MATCH, RETURN, etc.)
                is_read_query = statement.strip().upper().startswith(('MATCH', 'RETURN', 'CALL', 'WITH'))
                
                # Execute the statement
                result = session.run(statement)
                
                # If it's a read query, collect the results
                if is_read_query:
                    # Convert result to a list of dictionaries
                    statement_results = [dict(record) for record in result]
                    results.extend(statement_results)
                    
            if results:
                print(f"Query returned {len(results)} records")
            else:
                print("Query executed successfully")
                
            return results
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        raise
    finally:
        driver.close()


In [72]:
def get_detailed_schema():
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
    
    try:
        with driver.session() as session:
            # Get all node labels
            labels_query = "CALL db.labels()"
            labels_result = session.run(labels_query)
            labels = [record["label"] for record in labels_result]
            
            # Get all relationship types
            rel_types_query = "CALL db.relationshipTypes()"
            rel_types_result = session.run(rel_types_query)
            rel_types = [record["relationshipType"] for record in rel_types_result]
            
            # Get node properties for each label
            node_properties = {}
            for label in labels:
                properties_query = f"""
                MATCH (n:{label})
                WITH n LIMIT 1
                RETURN keys(n) as properties
                """
                properties_result = session.run(properties_query)
                properties = [record["properties"] for record in properties_result]
                if properties and properties[0]:
                    node_properties[label] = properties[0]
                else:
                    node_properties[label] = []
            
            # Get existing nodes with their properties
            existing_nodes = {}
            for label in labels:
                nodes_query = f"""
                MATCH (n:{label})
                RETURN n
                """
                nodes_result = session.run(nodes_query)
                nodes = [dict(record["n"]) for record in nodes_result]
                if nodes:
                    existing_nodes[label] = nodes
            
            # Get relationship properties and structure
            relationship_details = []
            for rel_type in rel_types:
                rel_query = f"""
                MATCH (a)-[r:{rel_type}]->(b)
                WITH a, r, b LIMIT 10
                RETURN labels(a) as source_labels, 
                       labels(b) as target_labels, 
                       type(r) as relationship_type,
                       keys(r) as relationship_properties,
                       a.name as source_name,
                       b.name as target_name
                """
                rel_result = session.run(rel_query)
                for record in rel_result:
                    relationship_details.append({
                        "source_labels": record["source_labels"],
                        "target_labels": record["target_labels"],
                        "relationship_type": record["relationship_type"],
                        "relationship_properties": record["relationship_properties"],
                        "source_name": record["source_name"],
                        "target_name": record["target_name"]
                    })
            
            return {
                "node_labels": labels,
                "relationship_types": rel_types,
                "node_properties": node_properties,
                "existing_nodes": existing_nodes,
                "relationship_details": relationship_details
            }
    finally:
        driver.close()

In [73]:
class CypherQuery(BaseModel):
    query: List[str]

def generate_cypher_query(input: str) -> str:
        """
        Generate content for a specific outline section and store it in the database.
        
        Args:
            input: The input parameters for section content generation
            
        Returns:
            The generated section content
        """
        try:
            model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0).with_structured_output(CypherQuery)
            
            # Get previously generated content if available
            
            prompt = ChatPromptTemplate.from_messages(
                    [
                        ("system", """
                            You are an expert in Neo4j and Cypher query language. Generate a list of valid Cypher queries to create 
                            a graph database for a story with characters, locations, items, and relationships.
                            
                            CRITICAL RULES FOR GENERATING VALID CYPHER QUERIES:
                            
                            1. ALWAYS define variables before using them in the SAME statement
                            2. NEVER use shorthand variable names like 'p', 'd', 'l' - use descriptive names
                            3. NEVER store maps/dictionaries as property values - Neo4j only allows primitive types (strings, numbers, booleans) or arrays of primitive types
                            
                            CORRECT PROPERTY EXAMPLES:
                            ✓ SET character.name = 'Alex'  // String
                            ✓ SET character.age = 30  // Number
                            ✓ SET character.active = true  // Boolean
                            ✓ SET character.skills = ['climbing', 'swimming']  // Array of strings
                            
                            INCORRECT PROPERTY EXAMPLES (DO NOT USE THESE PATTERNS):
                            ✗ SET character.attributes = {{courage: 8, intelligence: 7}}  // Map/dictionary not allowed
                            ✗ SET p.age = 50  // Variable 'p' not defined
                            
                            For complex attributes, store each attribute as a separate property.
                            
                            Make sure to:
                            1. Create nodes with appropriate labels (Character, Location, Item, etc.)
                            2. Set properties for each node (name, description, attributes as separate properties)
                            3. Create relationships between nodes (LOCATED_AT, CONNECTS_TO, HAS_ITEM, etc.)
                            4. Use semicolons to separate statements
                            5. Ensure each statement is complete and can run independently
                            6. Always define variables before using them
                        """),
                        ("user", """
                            Generate a Cypher query to create the story initial state for this treasure hunt:
                            {input}
                            
                            Remember to never use maps/dictionaries as property values and always define variables before using them.
                        """),
                    ]
                ) 
            
           
            chain = {
                "input": lambda x: input,
            } | prompt | model
            
            return chain
        except Exception as e:
            print(f"Error generating section content: {str(e)}")
            raise

In [77]:

def generate_cypher_query_get_context(input: str) -> str:
    """
    Generate content for a specific outline section and store it in the database.
    
    Args:
        input: The input parameters for section content generation
        
    Returns:
        The generated section content
    """
    try:
        model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0).with_structured_output(CypherQuery)
        
        # Get previously generated content if available
            
        prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", """
                        You are an expert in Neo4j and Cypher query language. Generate a list of valid Cypher queries to get the data you need to get the context for the next section of the story.
                        Reference the schema of the graph database to create the query.
                        SCHEMA:
                        {schema}
                    """),
                    ("user", """
                        Next section of the story:
                        {input}
                    """),
                ]
            )
        
        
        chain = {
            "input": lambda x: input,
            "schema": lambda x: get_detailed_schema()
        } | prompt | model
        
        return chain
    except Exception as e:
        print(f"Error generating section content: {str(e)}")
        raise

In [78]:
def generate_cypher_update_queries(input: str) -> List[str]:
    """Generate Cypher queries to update the graph based on new story content"""
    
    # Initialize the LLM
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0).with_structured_output(CypherQuery)
    
    # Create a prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are an expert in Neo4j and story analysis. Your task is to analyze a new section of a story 
        and generate Cypher queries to update a Neo4j graph database with new elements from the story.
        
        The existing database has the following schema:
        {schema}
        
        CRITICAL RULES FOR GENERATING VALID CYPHER QUERIES:
        
        1. ALWAYS define variables before using them in the SAME statement
        2. NEVER use standalone SET statements - always include a MATCH clause first
        3. NEVER use shorthand variable names like 'p', 'd', 'l' - use descriptive names like 'professor', 'doctor', 'harbor'
        4. ALWAYS use full, complete queries that can run independently
        
        CORRECT EXAMPLES:
        ✓ MATCH (professor:Character {{name: 'Professor Reed'}}) SET professor.age = 50
        ✓ MATCH (harbor:Location {{name: 'Saltwater Harbor'}}), (lighthouse:Location {{name: 'Old Lighthouse'}}) MERGE (harbor)-[:CONNECTS_TO]->(lighthouse)
        
        INCORRECT EXAMPLES (DO NOT USE THESE PATTERNS):
        ✗ SET p.age = 50  // Variable 'p' not defined
        ✗ MERGE (l)-[:CONNECTS_TO]->(lighthouse)  // Variables not defined
        
        For the new story section, identify:
        1. New characters, locations, and items to add
        2. Updated information about existing entities
        3. New relationships between entities
        
        
        Use the following node labels:
        - Character (for people)
        - Location (for places)
        - Item (for objects)
        
        Use appropriate relationship types like:
        - KNOWS, FRIENDS_WITH, ENEMIES_WITH (between characters)
        - LOCATED_AT (character at location)
        - CONNECTS_TO (between locations)
        - HAS_ITEM (character possesses item)
        - PART_OF (item is part of another item)
        
        Return a list of Cypher queries, each as a complete statement with properly defined variables.
        """),
        ("user", """
        Here is the new story section:
        {input}
        
        Generate Cypher queries to update the graph database with new information from this section.
        Remember to always define variables before using them, and never use standalone SET statements.
        """),
    ])
    
    # Execute the chain
    chain = {
        "schema": lambda x: get_detailed_schema(),
        "input": lambda x: input
    } | prompt | model
    
    return chain

In [85]:
cypher_chain_create_initial_state = generate_cypher_query(init_state)
cypher_query_create_initial_state = cypher_chain_create_initial_state.invoke(init_state)
execute_cypher_query(cypher_query_create_initial_state.query)

Query executed successfully


[]

In [86]:
cypher_chain_get_context = generate_cypher_query_get_context(init_state)
cypher_query_get_context = cypher_chain_get_context.invoke(init_state)
retrieved_context = execute_cypher_query(cypher_query_get_context.query)

Query returned 9 records


In [82]:
retrieved_context

[]

In [91]:
cypher_chain_update_state = generate_cypher_update_queries(second_state)
cypher_query_update_state = cypher_chain_update_state.invoke(second_state)
updated_state = execute_cypher_query(cypher_query_update_state.query)

Query executed successfully


In [84]:
updated_state

[]

In [62]:
updated_state

[]

In [4]:
init_state = """
The Lost Crown of Eldoria
Alex stands in the quaint coastal village of Brookhaven, surrounded by thatched cottages and the scent of salty sea air. An aspiring treasure hunter, Alex has arrived following rumors of the legendary Lost Crown of Eldoria—an ancient artifact said to grant its bearer immense wisdom and power.
In Alex's possession are just a few simple items: a weathered map showing the region but missing key locations, a reliable brass compass, and a water bottle for the journey ahead. The village is peaceful, with fishing boats bobbing in the harbor and the imposing stone buildings of Eldoria University visible on the nearby hill.
Professor Reed, a renowned historian at the university, might know more about the crown. Captain Morgan, a gruff but experienced sailor, has a boat that could take travelers to the mysterious Crescent Isle where some believe part of the treasure might be hidden. Meanwhile, whispers around the village speak of a thief known only as Raven who may have already stolen an artifact connected to the crown.
The locals seem friendly enough, though they regard the newcomer with curiosity. An old fisherman sits by the docks, perhaps with stories to tell, and there's something glinting in the sand—an old coin that might be worth examining.
From Brookhaven, paths lead to several locations: the dense Whispering Woods with its winding forest path, the bustling Saltwater Harbor, and the scholarly halls of Eldoria University. Each direction offers potential clues about the Lost Crown, which is said to be broken into four fragments scattered across the land.
The quest has not truly begun yet. The crown's history remains shrouded in mystery, the locations of its fragments unknown. Knowledge, allies, and resources must be gathered to piece together this ancient treasure before others claim it for themselves.
The morning sun rises higher in the sky as Alex contemplates which path to choose first on this adventure to find the Lost Crown of Eldoria.
From Brookhaven, paths lead to several locations: the dense Whispering Woods with its winding forest path, the bustling Saltwater Harbor, and the scholarly halls of Eldoria University. Each direction offers potential clues about the Lost Crown, which is said to be broken into four fragments scattered across the land.
The quest has not truly begun yet. The crown's history remains shrouded in mystery, the locations of its fragments unknown. Knowledge, allies, and resources must be gathered to piece together this ancient treasure before others claim it for themselves.
The morning sun rises higher in the sky as Alex contemplates which path to choose first on this adventure to find the Lost Crown of Eldoria."""

In [88]:
second_state = """
Professor reed found a gun and shot alex in the chest.
The University's Secret
Alex decides to begin the search at Eldoria University, drawn by the promise of knowledge and the expertise of Professor Reed. The cobblestone path winds uphill, leading to an impressive collection of stone buildings covered in ivy. Students hurry across the courtyard, arms full of ancient texts and scrolls.
Inside the main hall, Alex finds directions to Professor Reed's office in the Department of Historical Artifacts. The professor's door stands slightly ajar, revealing a cluttered space filled with maps, books, and curious objects from distant lands.
"Professor Reed?" Alex calls, knocking gently on the door frame.
A woman in her fifties looks up from behind a large desk, her silver-streaked hair pulled back in a practical bun. Wire-rimmed glasses perch on her nose, and her eyes light up with interest at the sight of a visitor.
"Yes? Come in, come in. I don't believe we've met before." She gestures to a chair across from her desk, pushing aside a stack of papers to make eye contact.
Alex explains the purpose of the visit—the search for the Lost Crown of Eldoria—and shows the weathered map. Professor Reed's expression shifts from polite interest to intense focus.
"The Lost Crown... not many people know of its existence anymore." She studies Alex carefully. "What makes you think it's more than just a legend?"
After hearing Alex's reasoning, Professor Reed nods slowly and moves to a locked cabinet in the corner of her office. She retrieves an ancient leather-bound book with faded gold lettering.
"This is the Journal of Mariner Eldon, who claimed to have seen the crown intact over two centuries ago. According to his writings, the crown was split into four pieces by the last ruler of Eldoria to prevent it from falling into the wrong hands."
As she opens the journal, a small key falls from between its pages. Professor Reed looks surprised.
"I've read this journal dozens of times, but I've never seen this key before."
The key is small and ornate, made of tarnished silver with a peculiar symbol etched into its handle—a crescent moon intersected by a straight line.
"There's someone else you should meet," Professor Reed says, carefully placing the key in Alex's palm. "Dr. Lydia Blackwood teaches Ancient Languages in the east wing. She's been researching Eldorian symbols for years. This marking might mean something to her."
As Alex leaves the professor's office, a hooded figure slips away from the shadows of the hallway. The figure moves with practiced stealth toward the university's rear exit, clutching what appears to be a small notebook.
The east wing of the university is quieter, with fewer students. Dr. Blackwood's office door is closed, but a melodic humming can be heard from within. When Alex knocks, the humming stops abruptly.
"Enter at your own risk," calls a cheerful voice.
Dr. Lydia Blackwood is not what Alex expected. Young, perhaps in her early thirties, with bright blue hair and tattoos of ancient symbols visible on her forearms. Her office is even more chaotic than Professor Reed's, with walls covered in charts of symbols and languages.
"Professor Reed sent me," Alex begins, holding out the key. "She thought you might know what this symbol means."
Dr. Blackwood's eyes widen as she takes the key, examining it under a magnifying glass.
"Where did you find this?" she whispers, her voice suddenly serious. "This is the symbol of the Guardians of Eldoria—a secret society sworn to protect the crown's fragments." Ssworn to protect the crown's fragments." She looks up at Alex with newfound intensity. "And if you have this key, then you've just become part of something much bigger than a treasure hunt."
She pulls a large map from beneath a pile of books and spreads it across her desk. It shows the region in much greater detail than Alex's weathered map.
"This key opens a hidden chamber beneath the Old Lighthouse at the edge of Saltwater Harbor. According to legend, it contains a device that can help locate the crown fragments." She taps a location on the map. "But be careful. If Professor Reed found this key, others may know about it too. The Guardians aren't the only ones interested in the crown."
As if to confirm her warning, a distant crash echoes from the direction of Professor Reed's office, followed by shouts of alarm.
"You should go," Dr. Blackwood says urgently, rolling up the detailed map and thrusting it into Alex's hands. "Take the back stairs. I'll check on Professor Reed."
As Alex hurries down the back staircase, the university bell begins to toll—not the usual hourly chime, but a rapid, alarming sequence that sends students and faculty rushing into the corridors.
Outside, the sky has darkened with approaching storm clouds, and the windcarries the scent of rain. The path back to Brookhaven lies ahead, but so does the road to Saltwater Harbor and the mysterious lighthouse with its hidden chamber.
The key feels heavy in Alex's pocket, a new weight of responsibility. The treasure hunt has truly begun, and already it seems more dangerous—and more important—than Alex had imagined.
"""

In [90]:
second_state = "lydia blackwood found a gun and shot dr. reed in the chest."